<div align="right"><img src="logo-64x64.png" height="32" width="32"></div>

<p><h1>Heavy-Duty pricing of <br> Fixed Income financial contracts <br> with Julia</h1></p>
<hr>
<p><b>Felipe Noronha (<a href="https://github.com/felipenoris">@felipenoris</a>)</b></p>

# Disclaimer

The views expressed here are solely those of the author and do not in any way represent the views of the *Brazilian Development Bank*.

# My Background

* Computer nerd

* Interested in finance

* Current job as Market Risk Manager

    * define and implement pricing models
    
    * execute pricing routines

* Julia user for about 2 years

# The Problem

* price a big portfolio of fixed income contracts (Credit Portfolio)

* ... fast! 

* 2.4 million contracts

* 78 million cashflows

* 17GB of uncompressed raw data in CSV format

* more than US$ 150 billion in book value (2016 public balance sheet)

# What was available

* text editor (no dev tools)

* 8-core 2GHz Linux server running Jupyter (see https://github.com/felipenoris/math-server-docker)

# The Results

**20 minutes to build the database from CSV files** (done once)


*  17GB CSVs shrink to 6GB CSVs after removing unused columns


*  6GB CSVs shrink to 4GB julia compressed native files


* Still a lot of room for improvement

### **3.5 minutes for the pricing routine**


* from an empty julia session to the price results for each contract

 **10x** improvement comparing to corporate systems

# not possible without Julia

# Ideas to tackle the problem

* minimize IO operations by using memory buffers (memoize-like)


* make use of Julia's parallel computation features


* minimize memory allocation by using iterators instead of vectors


* minimize indirection by making use of immutable types


* transform CSV input files into some efficient file format for reading

# Solution Components


**InterestRates.jl**


**BusinessDays.jl**


**Market Data module**: database of historical prices


**Contract module** : provides a julia type for a Fixed Income contract


**Pricing module** : pricing logic

# InterestRates.jl

# Some theory on pricing Fixed Income contracts

```
     t=0           t=T
     
                    N
     ┌──────────────┘
     P
```



In general: $ P = N \times \text{discountfactor}(r, T) $

But, given $r$ and $T$, you must know the convention used to:


* how to count time


* how the function `discountfactor` is defined

# DayCountConvention


* *Actual360* : `(D2 - D1) / 360`


* *Actual365* : `(D2 - D1) / 365`


* *BDays252* : `bdays(D1, D2) / 252`, where `bdays` is the business days between `D1` and `D2` from [BusinessDays.jl package]

# CompoundingType

Defines how `discountfactor` is implemented.


* *ContinuousCompounding*
$$ exp(-rt) $$


* *SimpleCompounding*
$$\frac{1}{(1 + rt)}$$


* *ExponentialCompounding*
$$\frac{1}{(1+r)^t}$$

# Term Structure

A Term Structure of Interest Rates, also known as *zero-coupon curve*, is a function `f(t) → y` that maps a given maturity `t` onto the yield `y` of a bond that matures at `t` and pays no coupons (*zero-coupon bond*).

It's not feasible to observe prices for each possible maturity. We can observe only a set of discrete data points of the yield curve. Therefore, in order to determine the entire term structure, one must choose an interpolation method, or a term structure model.



# Curve Methods provided by InterestRates.jl

* `<<CurveMethod>>`
	* `<<Interpolation>>`
		* `<<DiscountFactorInterpolation>>`
			* `CubicSplineOnDiscountFactors`
			* `FlatForward`
		* `<<RateInterpolation>>`
			* `CubicSplineOnRates`
			* `Linear`
			* `StepFunction`
		* `CompositeInterpolation`
	* `<<Parametric>>`
		* `NelsonSiegel`
		* `Svensson`

In [1]:
curve_date = Date(2017,3,2)
days_to_maturity = [ 22, 83, 147, 208, 269, 332, 396, 458, 519, 581, 711, 834]
rates = [ 0.121875, 0.11359 , 0.10714 , 0.10255 , 0.100527, 0.09935 , 0.09859 , 0.098407, 0.098737, 0.099036, 0.099909, 0.101135];

In [2]:
# Pkg.add("InterestRates"); Pkg.add("BusinessDays")
using InterestRates, BusinessDays
ir = InterestRates
method = ir.CompositeInterpolation(ir.StepFunction(), ir.CubicSplineOnRates(), ir.FlatForward())
curva_ltn = ir.IRCurve("LTN", ir.BDays252(:Brazil), ir.ExponentialCompounding(), method, curve_date, days_to_maturity, rates);

In [3]:
using Plots; gr()
x_axis = collect((curve_date+Dates.Day(1)):Dates.Day(1):Date(2022,2,1))
y_axis = ir.zero_rate(curva_ltn, x_axis)
plot(x_axis, y_axis, xlabel="maturity", ylabel="rates", ylims=(0, max(rates...)+0.05), legend=:none, linewidth=2)
plot!([ advancebdays(:Brazil, curve_date, d) for d in days_to_maturity ], rates, markershape=:circle, linewidth=0)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2018-01-01 
 
 
 2019-01-01 
 
 
 2020-01-01 
 
 
 2021-01-01 
 
 
 2022-01-01 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 maturity 
 
 
 rates 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 55.0141,112.785 55.3045,112.785 55.5948,112.785 55.8852,112.785 56.1755,112.785 56.4659,112.785 56.7562,112.785 57.0466,112.785 57.3369,112.785 57.6273,112.785 
 57.9176,112.785 58.208,112.785 58.4983,112.785 58.7887,112.785 59.079,112.785 59.3694,112.785 59.6597,112.785 59.9501,112.785 60.2404,112.785 60.5308,112.785 
 60.8211,112.785 61.1115,112.785 61.4018,112.785 61.6922,112.785 61.9825,112.785 62.2729,112.785 62.5632,112.785 62.8536,112.785 63.1439,112.785 63.4343,112.785 
 63.7246,112.785 64.015,112.785 64.3053,113.087 64.5956,113.388 64.886,113.69 65.1763,113.992 65.4667,114.294 65.757,114.294 66.0474,114.294 66.3377,114.595 
 66.6281,114.897 66.9184,115.198 67.2088,115.499 67.4991,115.499 67.7895,115.499 68.0798,115.499 68.3702,115.8 68.6605,116.1 68.9509,116.4 69.2412,116.7 
 69.5316,116.7 69.8219,116.7 70.1123,116.7 70.4026,117 70.693,117.299 70.9833,117.597 71.2737,117.896 71.564,118.193 71.8544,118.193 72.1447,118.193 
 72.4351,118.193 72.7254,118.491 73.0158,118.788 73.3061,119.084 73.5965,119.379 73.8868,119.379 74.1772,119.379 74.4675,119.674 74.7579,119.969 75.0482,120.263 
 75.3386,120.556 75.6289,120.848 75.9193,120.848 76.2096,120.848 76.5,121.14 76.7903,121.43 77.0807,121.721 77.371,122.01 77.6614,122.298 77.9517,122.298 
 78.2421,122.298 78.5324,122.586 78.8228,122.872 79.1131,123.158 79.4035,123.443 79.6938,123.726 79.9842,123.726 80.2745,123.726 80.5649,124.009 80.8552,124.291 
 81.1456,124.571 81.4359,124.851 81.7263,125.129 82.0166,125.129 82.307,125.129 82.5973,125.406 82.8877,125.682 83.178,125.957 83.4684,126.231 83.7587,126.503 
 84.0491,126.503 84.3394,126.503 84.6298,126.774 84.9201,127.044 85.2105,127.312 85.5008,127.312 85.7912,127.579 86.0815,127.579 86.3719,127.579 86.6622,127.845 
 86.9526,128.109 87.2429,128.372 87.5333,128.633 87.8236,128.893 88.114,128.893 88.4043,128.893 88.6947,129.151 88.985,129.408 89.2754,129.663 89.5657,129.916 
 89.8561,130.168 90.1464,130.168 90.4368,130.168 90.7271,130.418 91.0175,130.667 91.3078,130.914 91.5982,131.159 91.8885,131.403 92.1789,131.403 92.4692,131.403 
 92.7596,131.645 93.0499,131.885 93.3403,132.124 93.6306,132.362 93.921,132.598 94.2113,132.598 94.5017,132.598 94.792,132.832 95.0824,133.065 95.3727,133.297 
 95.6631,133.527 95.9534,133.756 96.2438,133.756 96.5341,133.756 96.8245,133.984 97.1148,134.21 97.4052,134.435 97.6955,134.658 97.9859,134.881 98.2762,134.881 
 98.5665,134.881 98.8569,135.102 99.1472,135.322 99.4376,135.54 99.7279,135.757 100.018,135.974 100.309,135.974 100.599,135.974 100.889,136.189 101.18,136.402 
 101.47,136.615 101.76,136.827 102.051,137.037 102.341,137.037 102.631,137.037 102.922,137.247 103.212,137.455 103.502,137.663 103.793,137.869 104.083,138.075 
 104.374,138.075 104.664,138.075 104.954,138.279 105.245,138.483 105.535,138.686 105.825,138.887 106.116,139.088 106.406,139.088 106.696,139.088 106.987,139.288 
 107.277,139.488 107.567,139.686 107.858,139.884 108.148,140.081 108.438,140.081 108.729,140.081 109.019,140.277 109.309,140.472 109.6,140.667 109.89,140.667 
 110.181,140.861 110.471,140.861 110.761,140.861 111.052,141.054 111.342,141.247 111.632,141.439 111.923,141.63 112.213,141.821 112.503,141.821 112.794,141.821 
 113.084,142.011 113.374,142.201 113.665,142.39 113.955,142.579 114.245,142.767 114.536,142.767 114.826,142.767 115.116,142.955 115.407,143.142 115.697,143.329 
 115.988,143.516 116.278,143.702 116.568,143.702 116.859,143.702 117.149,143.887 117.439,144.073 117.73,144.258 118.02,144.442 118.31,144.626 118.601,144.626 
 118.891,144.626 119.181,144.81 119.472,144.993 119.762,145.175 120.052,145.175 120.343,145.357 120.633,1

# BusinessDays.jl

In [4]:
using BusinessDays
bdays(:Brazil, Date(2017,6,22), Date(2017,6,26))

2 days

In [5]:
using BusinessDays
const bd = BusinessDays
d0 = Date(2015, 06, 29) ; d1 = Date(2100, 12, 20)
cal = bd.Brazil()

@elapsed bd.initcache(cal)

0.163521294

In [6]:
# this same benchmark takes 38 minutes to complete on QuantLib
@elapsed for i in 1:1000000 bdays(cal, d0, d1) end

0.540091417

# Pricing Design

**`price(model, contract)`**

* **`model`** has a **type** for *multiple-dispatch* to the correct price implementation

* **`model`** has all market data / historical data needed for the **price** method. There's no IO operation when executing the **price** method.

* **contract** is a data structure for the contract definition. 

## Pricing Steps

```julia
# contract search
contract = get_contract(db, code)

# get a unique identifier for the pricing model
model_key = infer_pricing_model_key(pricing_date, contract)

# creates an instance of the model.
# Connects to a database to retrieve market data (IO operation)
# This is done only once for each distinct value of model_key
model = get_pricing_model(conn, model_key)

# pricing formula
price(model, contract)
```

For fixed contracts, we can use the contract cash-flow directly, if available.

```julia
function price(model::FixedBond, c::Contract)
	mtm= 0.0

	for cf in c.cashflows		
		if p.value != 0.0
            mtm += p.valu 
            * discountfactor(model.curve_riskfree, cf.date) 
            * discountfactor(model.curve_spread, cf.date)
		end
	end
	return mtm * model.currency_spot_value
end
```

For floating-rate contracts, we must project cashflow using interest rate curves

```julia
function price(model::FloatingRate, c::Contract)
    mtm = 0.0

    for (dt, v) in CashFlowIterator(model, c)
		if v != 0.0
			mtm += v 
            * discountfactor(model.curve_riskfree, dtcp_ajustado) 
            * discountfactor(model.curve_spread, dtcp_ajustado)
		end
	end

	return mtm * model.currency_spot_value
end
```